---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 
          'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 
          'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 
          'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey',
          'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 
          'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 
          'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska',
          'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 
          'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire',
          'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
# states

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    university_towns = pd.read_fwf("university_towns.txt", header = None)
    
    regions_by_state = {}
    for name in university_towns.iloc[:,0]:
        if '[edit]' in name:
            key = name.replace('[edit]', '')
            regions_by_state[key] = []
        else:
            regions_by_state[key].append(re.sub(r' \(.*', '', name))
            
    final_df = pd.DataFrame()
    for key in regions_by_state.keys():
        temp = pd.DataFrame()
        temp['RegionName'] = regions_by_state[key]
        temp['State'] = [key for i in range(len(temp['RegionName']))]
        temp = temp[['State', 'RegionName']]
        final_df = pd.concat([final_df, temp])


    final_df.index = list(range(len(final_df)))
    
    return final_df

get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdp_df = pd.read_excel('gdplev.xls', skiprows = 7)
    gdp_df = gdp_df.iloc[:, [4,6]]
    gdp_df.columns = ['year_qtr', 'gdp']
    
    mask = gdp_df.year_qtr.apply(lambda x: True if (int(re.sub(r'q\d','',x)) >= 2000) else False)
    gdp_df = gdp_df.loc[mask,:]
    gdp_df.index = list(range(len(gdp_df)))
    
    
    for i in range(len(gdp_df)-2):
        if (gdp_df['gdp'][i] > gdp_df['gdp'][i+1]) & (gdp_df['gdp'][i+1] > gdp_df['gdp'][i+2]):
            if (i>0):
                if (gdp_df['gdp'][i-1] < gdp_df['gdp'][i]) & (gdp_df['gdp'][i] > gdp_df['gdp'][i+1]) & (gdp_df['gdp'][i+1] > gdp_df['gdp'][i+2]):
                    return(gdp_df['year_qtr'][i+1])
            
            return(gdp_df['year_qtr'][i])

get_recession_start()

'2008q3'

Example for the definitions: Suppose q1 < q2 > q3 > q4 > q5 < q6 < q7 < q8. Then, the recession start is q3, bottom is q5 and end is q7. Note that q2 is not a recession start as an upward surge is observed in this period (it is the quarter before the recession start in this case).



In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    gdp_df = pd.read_excel('gdplev.xls', skiprows = 7)
    gdp_df = gdp_df.iloc[:, [4,6]]
    gdp_df.columns = ['year_qtr', 'gdp']
    
    mask = gdp_df.year_qtr.apply(lambda x: True if (int(re.sub(r'q\d','',x)) >= 2000) else False)
    gdp_df = gdp_df.loc[mask,:]
    gdp_df.index = list(range(len(gdp_df)))
    
    recession_start = get_recession_start()
    i = gdp_df['year_qtr'].index[gdp_df['year_qtr']==recession_start].values[0] + 1
    
    while (True):
        ## if recession bottom is present
        if (gdp_df['gdp'][i] > gdp_df['gdp'][i+1]) & (gdp_df['gdp'][i+1] < gdp_df['gdp'][i+2]) & (gdp_df['gdp'][i+2] < gdp_df['gdp'][i+3]):
            return(gdp_df['year_qtr'][i+3])
        elif (gdp_df['gdp'][i] < gdp_df['gdp'][i+1]) & (gdp_df['gdp'][i+1] < gdp_df['gdp'][i+2]):
            return(gdp_df['year_qtr'][i+2])
        
        i+=1


get_recession_end()

'2009q4'

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
      
    gdp_df = pd.read_excel('gdplev.xls', skiprows = 7)
    gdp_df = gdp_df.iloc[:, [4,6]]
    gdp_df.columns = ['year_qtr', 'gdp']

    mask = gdp_df.year_qtr.apply(lambda x: True if (int(re.sub(r'q\d','',x)) >= 2000) else False)
    gdp_df = gdp_df.loc[mask,:]
    gdp_df.index = list(range(len(gdp_df)))


    recession_start = get_recession_start()
    i = gdp_df['year_qtr'].index[gdp_df['year_qtr']==recession_start].values[0] + 1
    
    while (True):
        ## if recession bottom is present
        if (gdp_df['gdp'][i] > gdp_df['gdp'][i+1]) & (gdp_df['gdp'][i+1] < gdp_df['gdp'][i+2]) & (gdp_df['gdp'][i+2] < gdp_df['gdp'][i+3]):
            return(gdp_df['year_qtr'][i+1])
        elif (gdp_df['gdp'][i] < gdp_df['gdp'][i+1]) & (gdp_df['gdp'][i+1] < gdp_df['gdp'][i+2]):
            return(gdp_df['year_qtr'][i])
        
        i+=1

get_recession_bottom()

'2009q2'

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    house = pd.read_csv("City_Zhvi_AllHomes.csv")
    house = house.set_index(['State', 'RegionName'])
    house = house.loc[:, house.columns[house.columns.str.contains(pat = 
                                                                  '20[0-9]{2}-[0-9]{2}', 
                                                                  regex = True)]]


    for year in range(2000, 2016):
        house[str(year) + 'q1'] = house[[str(year)+'-01', str(year)+'-02', str(year)+'-03']].apply(np.mean, axis = 1)
        house[str(year) + 'q2'] = house[[str(year)+'-04', str(year)+'-05', str(year)+'-06']].apply(np.mean, axis = 1)
        house[str(year) + 'q3'] = house[[str(year)+'-07', str(year)+'-08', str(year)+'-09']].apply(np.mean, axis = 1)
        house[str(year) + 'q4'] = house[[str(year)+'-10', str(year)+'-11', str(year)+'-12']].apply(np.mean, axis = 1)

    house = house.drop([str(x)+'-'+y for x in range(2000,2016) for y in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']] 
                       , axis = 1)

    house['2016'+'q1'] = house[['2016'+'-01', '2016'+'-02','2016'+'-03']].apply(np.mean, axis = 1)
    house['2016'+'q2'] = house[['2016'+'-04', '2016'+'-05','2016'+'-06']].apply(np.mean, axis = 1)
    house['2016'+'q3'] = house[['2016'+'-07', '2016'+'-08']].apply(np.mean, axis = 1)

    house = house.drop(['2016-'+x for x in ['01', '02', '03', '04', '05', '06', '07', '08']], axis = 1)
    
    return house

data = convert_housing_data_to_quarters()
data.head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
CA,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
IL,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
PA,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
AZ,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


data.head()

recession_start = get_recession_start()
recession_bottom = get_recession_bottom()
print("rece start = ",recession_start)
print("rece bottom = ", recession_bottom)

df = data.loc[:, [recession_start,recession_bottom]].reset_index()
df['State'] = df['State'].apply(lambda x: states[x])
df['Ratio'] = df[recession_start]/df[recession_bottom]
df = df[['State', 'RegionName', 'Ratio']]
df['combin'] = df['State'] + '|' + df['RegionName']
df.head()

df.shape

df.head()

university_towns_list = get_list_of_university_towns()
university_towns_list.head()

university_towns_list.shape

university_towns = pd.merge(university_towns_list, df, how = 'inner',
                            left_on=['State', 'RegionName'],right_on=['State', 'RegionName'])
university_towns.head()

non_university_towns = df.loc[~df['combin'].isin(university_towns['combin']),:]
non_university_towns.head()

non_university_towns.shape

university_towns = university_towns[['State', 'RegionName', 'Ratio']]
university_towns = university_towns.set_index(['State', 'RegionName'])
non_university_towns = non_university_towns[['State', 'RegionName', 'Ratio']]
non_university_towns = non_university_towns.set_index(['State', 'RegionName'])
university_towns.head()

non_university_towns.head()

np.sum(non_university_towns['Ratio'])

tstat, p_value = ttest_ind(university_towns['Ratio'].dropna(), non_university_towns['Ratio'].dropna())
tstat, p_value

different = True if (p_value < 0.01) else False

better = "university town" if (university_towns['Ratio'].mean() < 
                               non_university_towns['Ratio'].mean()) else "non-university town"


(different, p_value, better)

__Ho(Null hypothesis):__ 

__Ha(Alternate hypothesis):__ 

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()

    df = data.loc[:, [recession_start,recession_bottom]].reset_index()
    df['State'] = df['State'].apply(lambda x: states[x])
    df['Ratio'] = df[recession_start]/df[recession_bottom]
    df = df[['State', 'RegionName', 'Ratio']]
    df['combin'] = df['State'] + '|' + df['RegionName']

    university_towns_list = get_list_of_university_towns()

    # university towns with price ratio
    university_towns = pd.merge(university_towns_list, df, how = 'inner',
                            left_on=['State', 'RegionName'],right_on=['State', 'RegionName'])

    # non-university data
    non_university_towns = df.loc[~df['combin'].isin(university_towns['combin']),:]

    university_towns = university_towns[['State', 'RegionName', 'Ratio']]
    university_towns = university_towns.set_index(['State', 'RegionName'])
    non_university_towns = non_university_towns[['State', 'RegionName', 'Ratio']]
    non_university_towns = non_university_towns.set_index(['State', 'RegionName'])

    # t-test
    tstat, p_value = ttest_ind(university_towns['Ratio'].dropna(), non_university_towns['Ratio'].dropna())


    different = True if (p_value < 0.01) else False
    better = "university town" if (university_towns['Ratio'].mean() < 
                               non_university_towns['Ratio'].mean()) else "non-university town"


    return (different, p_value, better)


run_ttest()

(True, 0.0054964273536938875, 'university town')